In [13]:
from pykrx import stock
from tqdm import tqdm
import time
import pandas as pd
import FinanceDataReader as fdr
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

stocks = fdr.StockListing('KRX')['Symbol']

def date_to_str(df) :
    tmp = pd.DataFrame(df.index)
    print(len(tmp))
    for i in range(len(tmp)) :
        tmp.loc[i,'날짜'] = str(tmp.loc[i,'날짜'])[0:4]+str(tmp.loc[i,'날짜'])[5:7]+str(tmp.loc[i,'날짜'])[8:10]
    return tmp

def start_end_date(code) :
    try : 
        print(code)
        df2 = fdr.DataReader(code)
    except : 
        print(code)
        df2 = fdr.DataReader(code, exchange='KRX-DELISTING')
    if len(df2) == 0:
        print('0')
        return 0
    start_date = str(df2.index[0])[0:4]+str(df2.index[0])[5:7]+str(df2.index[0])[8:10]
    end_date = str(df2.index[-1])[0:4]+str(df2.index[-1])[5:7]+str(df2.index[-1])[8:10]
    if int(start_date) < int('20010101') :
        start_date = '20010101'
    return [start_date, end_date]

In [14]:
# 삼성전자, SK하이닉스로 2001.01.01~2021.10.11까지의 유효거래일 DF 생성
# 5133일로 동일
sk_df = stock.get_market_ohlcv_by_date("20110101", "20211011", "000660") #20211011
sm_df = stock.get_market_ohlcv_by_date("20110101", "20211011", "005930")
sk = date_to_str(sk_df)
sm = date_to_str(sm_df)
date_df = pd.merge(sm, sk, left_on='날짜', right_on='날짜', how='outer')
date_df_stamp = sk_df.index
print(sk_df.index)

2653
2653
DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2021-09-24', '2021-09-27', '2021-09-28', '2021-09-29',
               '2021-09-30', '2021-10-01', '2021-10-05', '2021-10-06',
               '2021-10-07', '2021-10-08'],
              dtype='datetime64[ns]', name='날짜', length=2653, freq=None)


In [15]:
# KOSDAQ

# 받아올 데이터 저장할 행열 생성

df = stock.get_index_price_change_by_ticker("20200520", "20200527", "KOSDAQ")
print(df)
import sys
mod = sys.modules[__name__]
index_list = df.index
for idx in range(len(df.index)) :
    setattr(mod, 'Kd_idx_{}'.format(idx), [])

print(index_list)


                        시가       종가       등락률          거래량            거래대금
지수명                                                                       
코스닥                 696.36   724.59  4.050781  10488319776  62986196230829
코스닥 150            1065.42  1102.57  3.490234    729479528  18619100922088
코스닥 150 정보기술        603.58   631.64  4.648438    268338653   5203201290465
코스닥 150 헬스케어       3450.56  3532.41  2.369141    135927364   7874689575610
코스닥 150 커뮤니케이션서비스  2037.30  2090.19  2.599609     25001250    816778277690
...                    ...      ...       ...          ...             ...
코스닥 소형주            2285.73  2387.53  4.449219   7150975272  24224003176345
코스닥 우량기업부          1312.10  1355.88  3.339844   1917808808  25514140717966
코스닥 벤처기업부          1494.98  1566.60  4.789062   1917465581   9194961598450
코스닥 중견기업부          1440.69  1507.13  4.609375   5538422300  21010396782694
코스닥 기술성장기업부        5395.23  5755.56  6.679688    348957489   6580691472020

[50 rows x 5 columns]
In

In [16]:
# 20010102 부터 하루씩 코스닥의 종류별 지수를 받아옴
 
for i in tqdm(range(len(date_df)-1)) :
    try : 
        df = stock.get_index_price_change_by_ticker(date_df.iloc[i].values[0], date_df.iloc[i+1].values[0], "KOSDAQ")
        time.sleep(1)
    except :
        continue
    for j in range(len(index_list)) :
        if len(getattr(mod, 'Kd_idx_{}'.format(j)))==0 :
            try : 
                tmp = pd.DataFrame(df.loc[index_list[j]])
                tmp = tmp.transpose()
                tmp.rename(index = {index_list[j] : date_df_stamp[i]}, inplace = True)
                setattr(mod, 'Kd_idx_{}'.format(j),tmp)
            except :
                continue
        else :
            try : 
                tmp = pd.DataFrame(df.loc[index_list[j]])
                tmp = tmp.transpose()
                tmp.rename(index = {index_list[j] : date_df_stamp[i]}, inplace = True)
                result = getattr(mod, 'Kd_idx_{}'.format(j)).append(tmp)
                setattr(mod, 'Kd_idx_{}'.format(j),result)
            except :
                continue

for j in tqdm(range(len(index_list))) :
    try :
        pd.DataFrame(getattr(mod, 'Kd_idx_{}'.format(j))).to_csv('Kd'+str(index_list[j])+'.csv', index = True, header = True)
    except :
        print (str(index_list[j]))


 44%|████▍     | 22/50 [00:00<00:00, 43.71it/s]

IT S/W & SVC
IT H/W


100%|██████████| 50/50 [00:01<00:00, 37.37it/s]


In [17]:
df = stock.get_index_price_change_by_ticker("20200520", "20200527", "KOSPI")
print(df)
import sys
mod = sys.modules[__name__]
index_list = df.index
for idx in range(len(df.index)) :
    setattr(mod, 'Kp_idx_{}'.format(idx), [])

print(index_list)

                       시가       종가       등락률         거래량            거래대금
지수명                                                                     
코스피               1980.61  2031.20  2.550781  4986224303  59536138219518
코스피 200            261.79   267.64  2.230469  1013842486  38405608497693
코스피 100           2008.94  2054.10  2.250000   641432999  33051548217828
코스피 50            1809.93  1852.32  2.339844   367982667  28238216528794
코스피 200 중소형주       777.16   802.42  3.250000   372409487   5354060279865
...                   ...      ...       ...         ...             ...
코스피 중형주           2079.18  2145.06  3.169922   966624765  12282601611076
코스피 소형주           1702.59  1755.27  3.089844  3235511719  11157724709356
코스피 200 비중상한 30%   261.79   267.64  2.230469  1013842486  38405608497693
코스피 200 비중상한 25%   259.64   266.00  2.449219  1013842486  38405608497693
코스피 200 비중상한 20%   253.16   259.95  2.679688  1013842486  38405608497693

[46 rows x 5 columns]
Index(['코스피', '코스피 200', '코스

In [18]:
# 20010102 부터 하루씩 코스닥의 종류별 지수를 받아옴
 
for i in tqdm(range(len(date_df)-1)) :
    try : 
        df = stock.get_index_price_change_by_ticker(date_df.iloc[i].values[0], date_df.iloc[i+1].values[0], "KOSPI")
        time.sleep(1)
    except :
        continue
    for j in range(len(index_list)) :
        if len(getattr(mod, 'Kp_idx_{}'.format(j)))==0 :
            try : 
                tmp = pd.DataFrame(df.loc[index_list[j]])
                tmp = tmp.transpose()
                tmp.rename(index = {index_list[j] : date_df_stamp[i]}, inplace = True)
                setattr(mod, 'Kp_idx_{}'.format(j),tmp)
            except :
                continue
        else :
            try : 
                tmp = pd.DataFrame(df.loc[index_list[j]])
                tmp = tmp.transpose()
                tmp.rename(index = {index_list[j] : date_df_stamp[i]}, inplace = True)
                result = getattr(mod, 'Kp_idx_{}'.format(j)).append(tmp)
                setattr(mod, 'Kp_idx_{}'.format(j),result)
            except :
                continue

for j in tqdm(range(len(index_list))) :
    try :
        pd.DataFrame(getattr(mod, 'Kp_idx_{}'.format(j))).to_csv('Kp'+str(index_list[j])+'.csv', index = True, header = True)
    except :
        print (str(index_list[j]))


 35%|███▍      | 16/46 [00:00<00:00, 44.84it/s]

코스피 200 철강/소재
코스피 200 에너지/화학


100%|██████████| 46/46 [00:01<00:00, 37.31it/s]
